array(['Russia_national_football_team'], dtype='<U29')

# L5: Information extraction

Information extraction (IE) is the task of identifying named entities and semantic relations between these entities in text data. In this lab we will focus on two sub-tasks in IE, **named entity recognition** (identifying mentions of entities) and **entity linking** (matching these mentions to entities in a knowledge base).

We start by loading spaCy:

In [148]:
import spacy
from spacy.gold import GoldParse

nlp = spacy.load('en_core_web_sm')


The data that we will be using has been tokenized following the conventions of the [Penn Treebank](ftp://ftp.cis.upenn.edu/pub/treebank/public_html/tokenization.html), and we need to prevent spaCy from using its own tokenizer on top of this. We therefore override spaCy&rsquo;s tokenizer with one that simply splits on space.

In [149]:
from spacy.tokens import Doc

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        return Doc(self.vocab, words=text.split(' '))

nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

## Data set

The main data set for this lab is a collection of news wire articles in which mentions of named entities have been annotated with page names from the [English Wikipedia](https://en.wikipedia.org/wiki/). The next code cell loads the training and the development parts of the data into Pandas data frames.

In [150]:
import bz2
import csv
import pandas as pd

with bz2.open('ner-train.tsv.bz2', 'rt',encoding="utf-8") as source:
    df_train = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE,encoding='utf-8')

with bz2.open('ner-dev.tsv.bz2', 'rt',encoding="utf-8") as source:
    df_dev = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE,encoding='utf-8')

Each row in these two data frames corresponds to one mention of a named entity and has five columns:

1. a unique identifier for the sentence containing the entity mention
2. the pre-tokenized sentence, with tokens separated by spaces
3. the start position of the token span containing the entity mention
4. the end position of the token span (exclusive, as in Python list indexing)
5. the entity label; either a Wikipedia page name or the generic label `--NME--`

The following cell prints the first five samples from the training data:

In [151]:
df_train.head()

,sentence_id,sentence,beg,end,label
0,0000-000,EU rejects German call to boycott British lamb .,0,1,--NME--
1,0000-000,EU rejects German call to boycott British lamb .,2,3,Germany
2,0000-000,EU rejects German call to boycott British lamb .,6,7,United_Kingdom
3,0000-001,Peter Blackburn,0,2,--NME--
4,0000-002,BRUSSELS 1996-08-22,0,1,Brussels


In this sample, we see that the first sentence is annotated with three entity mentions:

* the span 0–1 &lsquo;EU&rsquo; is annotated as a mention but only labelled with the generic `--NME--`
* the span 2–3 &lsquo;German&rsquo; is annotated with the page [Germany](http://en.wikipedia.org/wiki/Germany)
* the span 6–7 &lsquo;British&rsquo; is annotated with the page [United_Kingdom](http://en.wikipedia.org/wiki/United_Kingdom)

## Problem 1: Evaluation measures

To warm up, we ask you to write code to print the three measures that you will be using for evaluation:

In [152]:
def evaluation_report(gold, pred):
    """Print precision, recall, and F1 score.

    Args:
        gold: The set with the gold-standard values.
        pred: The set with the predicted values.
    
    Returns:
        Nothing, but prints the precision, recall, and F1 values computed
        based on the specified sets.
    """
    # TODO: Replace the next line with your own code
    precision=len(gold.intersection(pred))/len(pred)*100
    recall=len(gold.intersection(pred))/len(gold)*100
    F1=2*(recall*precision)/(precision+recall)
    print(precision,recall,F1)

To test your code, you can run the following cell:

In [153]:
evaluation_report(set(range(3)), set(range(5)))

60.0 100.0 75.0


This should give you a precision of 60%, a recall of 100%, and an F1-value of 75%.

## Problem 2: Span recognition

One of the first tasks that an information extraction system has to solve is to locate and classify (mentions of) named entities, such as persons and organizations. Here we will tackle the simpler task of recognizing **spans** of tokens that contain an entity mention, without the actual entity label.

The English language model in spaCy features a full-fledged [named entity recognizer](https://spacy.io/usage/linguistic-features#named-entities) that identifies a variety of entities, and can be updated with new entity types by the user. Your task in this problem is to evaluate the performance of this component when predicting entity spans in the development data.

Start by implementing a generator function that yields the gold-standard spans in a given data frame.

**Hint:** The Pandas method [`itertuples()`](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.itertuples.html) is useful when iterating over the rows in a DataFrame.

In [154]:
def gold_spans(df):
    """Yield the gold-standard mention spans in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        triples consisting of the sentence id, start position, and end
        position of each span.
    """
    # TODO: Replace the next line with your own code
    #yield
    for x in df.itertuples():
        yield x[1],x[3],x[4]

To test your code, you can count the spans yielded by your function. When called on the development data, you should get a total of 5,917 unique triples. The first triple and the last triple should be

    ('0946-000', 2, 3)
    ('1161-010', 1, 3)  

In [155]:
spans_dev_gold = set(gold_spans(df_dev))
print(len(spans_dev_gold))

5917


Your next task is to write code that calls spaCy to predict the named entities in the development data, and to evaluate the accuracy of these predictions in terms of precision, recall, and F1. Print these scores using the function that you wrote for Problem&nbsp;1.

In [156]:
# TODO: Write code here to run and evalp;uate the spaCy NER on the development data
ent_pred=[]
for x in df_dev.itertuples(index=False,name=None):
    doc = nlp(x[1])
    for ent in doc.ents:
        #print(ent)
        ent_pred.append((x[0],ent.start,ent.end))
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)
print(ent_pred)

 18), ('1152-016', 22, 24), ('1152-016', 25, 26), ('1152-016', 26, 27), ('1152-016', 1, 2), ('1152-016', 2, 3), ('1152-016', 8, 9), ('1152-016', 11, 12), ('1152-016', 14, 15), ('1152-016', 16, 17), ('1152-016', 17, 18), ('1152-016', 22, 24), ('1152-016', 25, 26), ('1152-016', 26, 27), ('1152-016', 1, 2), ('1152-016', 2, 3), ('1152-016', 8, 9), ('1152-016', 11, 12), ('1152-016', 14, 15), ('1152-016', 16, 17), ('1152-016', 17, 18), ('1152-016', 22, 24), ('1152-016', 25, 26), ('1152-016', 26, 27), ('1152-016', 1, 2), ('1152-016', 2, 3), ('1152-016', 8, 9), ('1152-016', 11, 12), ('1152-016', 14, 15), ('1152-016', 16, 17), ('1152-016', 17, 18), ('1152-016', 22, 24), ('1152-016', 25, 26), ('1152-016', 26, 27), ('1152-016', 1, 2), ('1152-016', 2, 3), ('1152-016', 8, 9), ('1152-016', 11, 12), ('1152-016', 14, 15), ('1152-016', 16, 17), ('1152-016', 17, 18), ('1152-016', 22, 24), ('1152-016', 25, 26), ('1152-016', 26, 27), ('1152-017', 3, 4), ('1152-017', 6, 7), ('1152-017', 9, 14), ('1152-017'

In [157]:
spans_dev_pred=set(ent_pred)
print(len(spans_dev_pred))
evaluation_report(spans_dev_gold,spans_dev_pred)

8034
51.53099327856609 69.9678891330066 59.35058418751344


## Problem 3: Error analysis

As you were able to see in Problem&nbsp;2, the span accuracy of the named entity recognizer is far from perfect. In particular, only slightly more than half of the predicted spans are correct according to the gold standard. Your next task is to analyse this result in more detail.

Here is a function that prints the false positives as well as the false negatives spans for a data frame, given a reference set of gold-standard spans and a candidate set of predicted spans.

In [158]:
from collections import defaultdict

def error_report(df, spans_gold, spans_pred):
    false_pos = defaultdict(list)
    for s, b, e in spans_pred - spans_gold:
        false_pos[s].append((b, e))
    false_neg = defaultdict(list)
    for s, b, e in spans_gold - spans_pred:
        false_neg[s].append((b, e))
    for row in df.drop_duplicates('sentence_id').itertuples():
        if row.sentence_id in false_pos or row.sentence_id in false_neg:
            print('Sentence:', row.sentence)
            for b, e in false_pos[row.sentence_id]:
                print('  FP:', ' '.join(row.sentence.split()[b:e]))
            for b, e in false_neg[row.sentence_id]:
                print('  FN:', ' '.join(row.sentence.split()[b:e]))

Use this function to inspect and analyse the errors that the automated prediction makes. Can you see any patterns? Base your analysis on the first 500 rows of the training data. Summarize your observations in a short text.

In [159]:
# TODO: Write code here to do your analysis
ent_pred=[]
print(1111)
for x in df_train.iloc[:500].itertuples():
    doc = nlp(x[2])
    for ent in doc.ents:
        ent_pred.append((x[1],ent.start,ent.end))
spans_train_pred=set(ent_pred)
error_report(df_train.iloc[:500],set(gold_spans(df_train)),spans_train_pred)

tified alarm through " dangerous generalisation . "
  FP: Spanish Farm
  FN: Spanish
Sentence: The EU 's scientific veterinary and multidisciplinary committees are due to re-examine the issue early next month and make recommendations to the senior veterinary officials .
  FP: early next month
Sentence: British farmers denied on Thursday there was any danger to human health from their sheep , but expressed concern that German government advice to consumers to avoid British lamb might influence consumers across Europe .
  FP: Thursday
Sentence: " What we have to be extremely careful of is how other countries are going to take Germany 's lead , " Welsh National Farmers ' Union ( NFU ) chairman John Lloyd Jones said on BBC radio .
  FP: BBC
  FN: BBC radio
Sentence: Bonn has led efforts to protect public health after consumer confidence collapsed in March after a British report suggested humans could contract an illness similar to mad cow disease by eating contaminated beef .
  FP: March
S

*TODO: Write a short text that summarises the errors that you observed*

False Positive is the wrongly predicted entities and we can see from previous result that most of the FPs are numbers, locations, time and date.

Now, use the insights from your error analysis to improve the automated prediction that you implemented in Problem&nbsp;2. While the best way to do this would be to [update spaCy&rsquo;s NER model](https://spacy.io/usage/linguistic-features#updating) using domain-specific training data, for this lab it suffices to write code to post-process the output produced by spaCy. You should be able to improve the F1 score from Problem&nbsp;2 by at last 15 percentage points.

In [160]:
# TODO: Write code here to improve the span prediction from Problem 2

ent_pred=[]
for x in df_dev.itertuples():
    doc = nlp(x[2])
    for ent in doc.ents:
        #print(ent)
        if ent.label_!=("CARDINAL" or "ORDINAL" or "DATE" or "TIME" or "PERCENT" or"PERSON" or "GPE" or "MONEY"):
            ent_pred.append((x[1],ent.start,ent.end))
print(ent_pred)

1', 26, 27), ('1152-012', 5, 6), ('1152-012', 11, 12), ('1152-012', 15, 16), ('1152-012', 5, 6), ('1152-012', 11, 12), ('1152-012', 15, 16), ('1152-012', 5, 6), ('1152-012', 11, 12), ('1152-012', 15, 16), ('1152-013', 0, 1), ('1152-013', 5, 6), ('1152-013', 9, 10), ('1152-013', 17, 18), ('1152-013', 0, 1), ('1152-013', 5, 6), ('1152-013', 9, 10), ('1152-013', 17, 18), ('1152-013', 0, 1), ('1152-013', 5, 6), ('1152-013', 9, 10), ('1152-013', 17, 18), ('1152-013', 0, 1), ('1152-013', 5, 6), ('1152-013', 9, 10), ('1152-013', 17, 18), ('1152-014', 5, 8), ('1152-014', 18, 19), ('1152-014', 23, 25), ('1152-014', 46, 47), ('1152-014', 49, 52), ('1152-014', 5, 8), ('1152-014', 18, 19), ('1152-014', 23, 25), ('1152-014', 46, 47), ('1152-014', 49, 52), ('1152-014', 5, 8), ('1152-014', 18, 19), ('1152-014', 23, 25), ('1152-014', 46, 47), ('1152-014', 49, 52), ('1152-014', 5, 8), ('1152-014', 18, 19), ('1152-014', 23, 25), ('1152-014', 46, 47), ('1152-014', 49, 52), ('1152-014', 5, 8), ('1152-014'

In [161]:
evaluation_report(spans_dev_gold,set(ent_pred))


66.4686044644291 69.95098867669427 68.16534914361002


Show that you achieve the performance goal by reporting the evaluation measures that you implemented in Problem&nbsp;1.

Before going on, we ask you to store the outputs of the improved named entity recognizer on the development data in a new data frame. This new frame should have the same layout as the original data frame for the development data that you loaded above, but should contain the *predicted* start and end positions for each token span, rather than the gold positions. As the `label` of each span, you can use the special value `--NME--`.

In [162]:
# TODO: Write code here to store the predicted spans in a new data frame
#df_dev[df_dev.sentence_id=="0946-002"].iloc[0].sentence
# sentence_id=[x[0] for x in ent_pred]
for i,e in enumerate(ent_pred):
    ent_pred[i]=(e[0],df_dev[df_dev.sentence_id==e[0]].iloc[0].sentence,e[1],e[2],"--NME--")
pred_df=pd.DataFrame(set(ent_pred))

## Problem 4: Entity linking

Now that we have a method for predicting mention spans, we turn to the task of **entity linking**, which amounts to predicting the knowledge base entity that is referenced by a given mention. In our case, for each span we want to predict the Wikipedia page that this mention references.

Start by extending the generator function that you implemented in Problem&nbsp;2 to labelled spans.

In [87]:
def gold_mentions(df):
    """Yield the gold-standard mentions in a data frame.

    Args:
        df: A data frame.

    Yields:
        The gold-standard mention spans in the specified data frame as
        quadruples consisting of the sentence id, start position, end
        position and entity label of each span.
    """
    # TODO: Replace the next line with your own code
    for x in df.itertuples():
        yield x[1],x[3],x[4],x[5]

A naive baseline for entity linking on our data set is to link each mention span to the Wikipedia page name that we get when we join the tokens in the span by underscores, as is standard in Wikipedia page names. Suppose, for example, that a span contains the two tokens

    Jimi Hendrix

The baseline Wikipedia page name for this span would be

    Jimi_Hendrix

Implement this naive baseline and evaluate its performance. Print the evaluation measures that you implemented in Problem&nbsp;1.

<div class="alert alert-warning">
    Here and in the remainder of this lab, you should base your entity predictions on the predicted spans that you computed in Problem&nbsp;3.
</div>

In [132]:
# TODO: Write code here to implement the baseline
naive_pred=[]

naive_pred=[]
for x in pred_df.itertuples():
    doc = nlp(x[2])
    for ent in doc.ents:
        tmp_str=[]
        for i in range(ent.start,ent.end):
            tmp_str.append(doc[i].text)
        
        tmp_str="_".join(tmp_str)
        naive_pred.append((x[1],ent.start,ent.end,tmp_str))
print(naive_pred)



1, 22, 'Brunswijk'), ('1003-003', 37, 39, '56_miles'), ('1003-003', 40, 42, '90_km'), ('1003-003', 45, 46, 'Paramaribo'), ('0960-011', 31, 32, "O'Brien"), ('0960-011', 34, 35, '65th'), ('0960-019', 3, 4, 'Wimbledon'), ('0960-019', 5, 8, 'two_years_ago'), ('0960-019', 12, 13, '28,000'), ('0960-019', 18, 19, 'year'), ('0960-019', 22, 25, 'the_All-England_Club'), ('0960-019', 34, 36, 'two_hour'), ('0960-019', 37, 39, '24_minute'), ('0960-019', 41, 42, 'Rios'), ('0974-002', 0, 1, 'Australia'), ('0990-005', 4, 6, 'Boris_Nikolayevich'), ('0990-005', 7, 8, 'Yeltsin'), ('0990-005', 9, 10, 'yesterday'), ('1009-007', 0, 3, 'The_Agriculture_Department'), ('1009-007', 4, 9, 'more_than_a_year_ago'), ('1009-007', 16, 17, 'Mexican'), ('1014-010', 8, 9, 'Texas'), ('1014-010', 10, 12, 'Ross_Perot'), ('1014-010', 13, 16, 'the_Reform_Party'), ('1014-010', 31, 36, 'the_Commission_on_Presidential_Debates'), ('1014-010', 48, 49, '1988'), ('1014-010', 50, 55, 'the_League_of_Women_Voters'), ('1130-010', 7, 9,

In [89]:
evaluation_report(set(gold_mentions(df_dev)),set(naive_pred))

20.097715568466924 25.7224945073517 22.564862861378803


## Problem 5: Extending the training data using the knowledge base

State-of-the-art approaches to entity linking exploit information in knowledge bases. In our case, where Wikipedia is the knowledge base, one particularly useful type of information are links to other Wikipedia pages. In particular, we can interpret the anchor texts (the highlighted texts that you click on) as mentions of the entities (pages) that they link to. This allows us to harvest long lists of mention–entity pairings.

The following cell loads a data frame summarizing anchor texts and page references harvested from the first paragraphs of the English Wikipedia. The data frame also contains all entity mentions in the training data (but not the development or the test data).

In [90]:
with bz2.open('kb.tsv.bz2', 'rt',encoding="utf-8") as source:
    df_kb = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

To understand what information is available in this data, the following cell shows the entry for the anchor text `Sweden`.

In [91]:
df_kb.loc[df_kb.mention == 'Sweden']

,mention,entity,prob
17436,Sweden,Sweden,0.985768
17437,Sweden,Sweden_national_football_team,0.014173
17438,Sweden,Sweden_men's_national_ice_hockey_team,0.000059


As you can see, each row of the data frame contains a pair $(m, e)$ of a mention $m$ and an entity $e$, as well as the conditional probability $P(e|m)$ for mention $m$ referring to entity $e$. These probabilities were estimated based on the frequencies of mention–entity pairs in the knowledge base. The example shows that the anchor text &lsquo;Sweden&rsquo; is most often used to refer to the entity [Sweden](http://en.wikipedia.org/wiki/Sweden), but in a few cases also to refer to Sweden&rsquo;s national football and ice hockey teams. Note that references are sorted in decreasing order of probability, so that the most probable pairing come first.

Implement an entity linking method that resolves each mention to the most probable entity in the data frame. If the mention is not included in the data frame, you can predict the generic label `--NME--`. Print the precision, recall, and F1 of your method using the function that you implemented for Problem&nbsp;1.

In [23]:
"abc dkej" =="abc"

False

In [133]:
# TODO: Write code here to implement the "most probable entity" method.
wiki_pred=[]
for x in pred_df.itertuples():
    doc = nlp(x[2])
    for ent in doc.ents:
        tmp_str=[]
        for i in range(ent.start,ent.end):
            tmp_str.append(doc[i].text)
        tmp_str=" ".join(tmp_str)
        tmp_df=df_kb.loc[df_kb.mention == tmp_str]
        if len(tmp_df)!=0:
            wiki_pred.append((x[1],ent.start,ent.end,tmp_df.iloc[0].entity))
        else:
            wiki_pred.append((x[1],ent.start,ent.end,"--NME--"))
print(wiki_pred)

E--'), ('0960-019', 37, 39, '--NME--'), ('0960-019', 41, 42, 'Marcelo_Ríos'), ('0974-002', 0, 1, 'Australia'), ('0990-005', 4, 6, '--NME--'), ('0990-005', 7, 8, 'Boris_Yeltsin'), ('0990-005', 9, 10, '--NME--'), ('1009-007', 0, 3, '--NME--'), ('1009-007', 4, 9, '--NME--'), ('1009-007', 16, 17, 'Mexico'), ('1014-010', 8, 9, 'Texas'), ('1014-010', 10, 12, 'Ross_Perot'), ('1014-010', 13, 16, '--NME--'), ('1014-010', 31, 36, '--NME--'), ('1014-010', 48, 49, '--NME--'), ('1014-010', 50, 55, 'League_of_Women_Voters'), ('1130-010', 7, 9, 'Prensa_Latina'), ('1130-010', 10, 11, '--NME--'), ('1130-010', 12, 13, '--NME--'), ('1152-007', 0, 1, 'Bill_Clinton'), ('1152-007', 3, 4, '--NME--'), ('1152-007', 7, 8, '--NME--'), ('1152-007', 11, 12, 'Persian_Gulf'), ('1152-007', 24, 25, 'Iraq'), ('1152-007', 27, 28, 'Kurds'), ('1152-007', 31, 32, 'Iraq'), ('1010-006', 1, 3, '--NME--'), ('1010-006', 4, 5, 'Chicago'), ('0984-010', 0, 1, '--NME--'), ('0984-010', 9, 11, '--NME--'), ('0984-010', 16, 17, 'United

In [134]:
evaluation_report(set(gold_mentions(df_dev)),set(wiki_pred))

41.95167040802853 53.69274970424202 47.10155670867309


## Problem 6: Context-sensitive disambiguation

Consider the entity mention &lsquo;Lincoln&rsquo;. The most probable entity for this mention turns out to be [Lincoln, Nebraska](http://en.wikipedia.org/Lincoln,_Nebraska); but in pages about American history, we would be better off to predict [Abraham Lincoln](http://en.wikipedia.org/Abraham_Lincoln). This suggests that we should try to disambiguate between different entity references based on the textual context on the page from which the mention was taken. Your task in this last problem is to implement this idea.

Set up a dictionary that contains, for each mention $m$ that can refer to more than one entity $e$, a separate Naive Bayes classifier that is trained to predict the correct entity $e$, given the textual context of the mention. As the prior probabilities of the classifier, choose the probabilities $P(e|m)$ that you used in Problem&nbsp;5. To let you estimate the context-specific probabilities, we have compiled a data set with mention contexts:

In [94]:
with bz2.open('contexts.tsv.bz2') as source:
    df_contexts = pd.read_csv(source, sep='\t', quoting=csv.QUOTE_NONE)

This data frame contains, for each ambiguous mention $m$ and each knowledge base entity $e$ to which this mention can refer, up to 100 randomly selected contexts in which $m$ is used to refer to $e$. For this data, a **context** is defined as the 5 tokens to the left and the 5 tokens to the right of the mention. Here are a few examples:

In [95]:
df_contexts.head()

,mention,entity,context
0,1970,UEFA_Champions_League,Cup twice the first in @ and the second in 1983
1,1970,FIFA_World_Cup,America 1975 and during the @ and 1978 World C...
2,1990 World Cup,1990_FIFA_World_Cup,Manolo represented Spain at the @
3,1990 World Cup,1990_FIFA_World_Cup,Hašek represented Czechoslovakia at the @ and ...
4,1990 World Cup,1990_FIFA_World_Cup,renovations in 1989 for the @ The present capa...


Note that, in each context, the position of the mention is indicated by the `@` symbol.

From this data frame, it is easy to select the data that you need to train the classifiers – the contexts and corresponding entities for all mentions. To illustrate this, the following cell shows how to select all contexts that belong to the mention &lsquo;Lincoln&rsquo;:

In [137]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from collections import Counter

classifiers={}
for w in set(df_contexts.mention):
    X=df_contexts.context[df_contexts.mention==w]
    y=df_contexts.entity[df_contexts.mention==w]
 
    c=Counter(y)
    prior = sorted(c.items(), key=lambda pair: (pair[0])) 
    prior= [p[1]/sum(c.values()) for p in prior]
    pipe=Pipeline([("vectorize",CountVectorizer()),("MultinomialNB",MultinomialNB(class_prior=prior))])

    pipe.fit(X,y)
    classifiers[w]=pipe


Implement the context-sensitive disambiguation method and evaluate its performance. Here are some more hints that may help you along the way:

**Hint 1:** The prior probabilities for a Naive Bayes classifier can be specified using the `class_prior` option. You will have to provide the probabilities in the same order as the alphabetically sorted class (entity) names.

**Hint 2:** Not all mentions in the knowledge base are ambiguous, and therefore not all mentions have context data. If a mention has only one possible entity, pick that one. If a mention has no entity at all, predict the `--NME--` label.

In [165]:
# TODO: Write code here to implement the context-sensitive disambiguation method
cs_pred=[]
for x in pred_df.itertuples():
    doc = nlp(x[2])
    for ent in doc.ents:
        tmp_str=[]
        for i in range(ent.start,ent.end):
            tmp_str.append(doc[i].text)
        tmp_str=" ".join(tmp_str)
        tmp_df=df_kb.loc[df_kb.mention == tmp_str]
        try:
            cs_pred.append((x[1],ent.start,ent.end,classifiers[tmp_str].predict([x[2]])[0]))
        except KeyError:
            if len(tmp_df)!=0:
                cs_pred.append((x[1],ent.start,ent.end,tmp_df.iloc[0].entity))
            else:
                cs_pred.append((x[1],ent.start,ent.end,"--NME--"))
print(cs_pred)


# cs_pred=[]
# for x in pred_df.itertuples():
#     doc = nlp(x[2])
#     for ent in doc.ents:
#         tmp_str=[]
#         for i in range(ent.start,ent.end):
#             tmp_str.append(doc[i].text)
#     tmp_str=" ".join(tmp_str)
#     try:
#         cs_pred.append((x[1],ent.start,ent.end,classifiers[tmp_str].predict([x[2]])[0]))
#     except KeyError:
#         cs_pred.append((x[1],ent.start,ent.end,"--NME--"))

# print(cs_pred)

--'), ('1031-016', 9, 10, 'China'), ('1119-004', 4, 5, 'Iraq'), ('1119-004', 14, 16, '--NME--'), ('0997-008', 0, 1, '--NME--'), ('0997-008', 2, 3, 'Boris_Yeltsin'), ('0997-008', 10, 12, '--NME--'), ('0997-008', 14, 15, 'Barvikha'), ('0997-008', 17, 18, 'Moscow'), ('1096-042', 3, 4, 'Serie_A'), ('1096-042', 5, 6, '--NME--'), ('1096-042', 7, 8, '--NME--'), ('1096-042', 8, 9, 'Run_(baseball)'), ('1096-042', 10, 11, 'UEFA_Euro_1996'), ('1160-004', 4, 6, 'Liam_Neeson'), ('1160-004', 7, 9, 'Julia_Roberts'), ('1160-004', 14, 16, 'Michael_Collins_(Irish_leader)'), ('1160-004', 17, 22, '--NME--'), ('1160-004', 28, 29, 'Republic_of_Ireland'), ('1160-004', 31, 34, '--NME--'), ('0958-052', 0, 2, 'San_Diego_Padres'), ('1072-018', 0, 2, 'South_Africa_national_rugby_union_team'), ('1072-018', 8, 9, '--NME--'), ('1072-018', 10, 12, '--NME--'), ('1072-018', 13, 14, '--NME--'), ('1072-018', 15, 17, 'Japie_Mulder'), ('1072-018', 18, 20, 'Joel_Stransky'), ('1072-018', 21, 22, '--NME--'), ('1072-018', 24, 

In [166]:
evaluation_report(set(gold_mentions(df_dev)),set(cs_pred))


39.85210616664466 51.00557715058307 44.744255003706456


You should expect to see a small (around 1&nbsp;unit) increase in both precision, recall, and F1.

**This was the last lab in the Text Mining course. Congratulations!**

<div class="alert alert-info">
    Please read the section ‘General information’ on the ‘Labs’ page of the course website before submitting this notebook!
</div>